# Radiation budget

*Paolo Davini (2024)*

This notebook aims at showing some properties of the Earth radiation budget, doing some calculation with climate model data. We will check some of the most important fields that constitues the surface and top-of-the-atmosphere (TOA) radiation budget, and we will do this by analyising the data from one climate simulation from the EC-Earth3 Global Climate Model. Again, data from the historical simulation will be analysed, but this time we will use yearly averaged data (again interpolated on a regular grid). Again, please note that this preprocessing operations have been all done with [CDO](https://code.mpimet.mpg.de/projects/cdo).

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

We exploit a powerful functionality for file loading of xarray, that allow us to load multiple file all together, namely the `xr.open_mfdataset()` method. This is very powerful since allow to load entire directory with a simple wildcard, but it might produce unexpected results if the files have not all the same structure. 
Concerning radiation data, in the CMIP6 naming convention, all radiation variables start with `r`. This is a bit though to follow, but has some advantages when understand how it works. 
- second letter: `l` or `s` meaning longwave or shortwave
- third letter: `u` or `d` meaning upwelling or downwelling
- fourth letter: `s` or `t` meaning at surface or at TOA
- bonus: `cs` meaning clear-sky radiation (we will get to that...)

In [ ]:
dataset = xr.open_mfdataset('ussp/r*EC-Earth*.nc', join="override", compat='override')
dataset

We can start having a look at the radiation at top of the atmosphere, which in model world means the last of the model vertical level. We can look at both shortwave and longwave radiation, but we will start by looking at the radiative longwave upwelling at TOA, i.e. `rlut`. 

In [ ]:
dataset['rlut'][-1,:,:].plot(vmin=150, vmax=330)

What am I selecting with the `[-1,:,:]`? 

In [ ]:
dataset['rlut'].sel(time=slice('2010-01-01', '2014-12-31')).mean(dim='time').plot(vmin=150, vmax=330)

- **What do you see? We can see different patterns, do you have an idea of what is this?**
- **Could you plot the other two fields at the TOA, `rsut` and `rsdt`? What they mean?**

## TOA net budget

For the Earth equilibrium temperature, and the climate dynamics that results, it is very important to assess its budget at the Top of the Atmosphere. This tells us how much energy is entering the system. At the TOA we will need 3 variables: the incoming solar radiation `rsdt`, the reflected solar radiation `rsut` and the outgoing longwave radiation `rlut`. Always double check signs!

In [ ]:
net_toa = dataset['rsdt']-dataset['rlut']-dataset['rsut']

Again, all the computation are very fast since everything is lazy. If you want to store the data into memory, for example if you need to run multiple plots, you can use the `.compute()` method

In [ ]:
toa_clim = net_toa.mean(dim='time').compute()

The plotting will be much faster, since the data has been already computed previously!

In [ ]:
toa_clim.plot()

- What is the consequence of having positive values at the Equator and negative at the poles? 

We could remark this in a cleaner doing what we name a **zonal** plot. We average along the longitudes!


In [ ]:
zonal_toa = toa_clim.mean(dim='lon')
zonal_toa.plot()

One nice thing we can do is to estimate the **poleward heat transport**, which can be obtained integrating the net toa radiation along the meridional direction. This is the amount of energy that the Earth is carrying from the equator to the pole. Considering that the model has a small bias, the more correct thing to do is to remove the mean so that the computation is correct.

This can be done by integrating it from South Pole to North Pole the TOA net radiation. The cumulative (i.e. the integral) can be done with the `numpy.cumsum()` function. However, we need to compute beforehand the areas of each latitudine circle. There are plenty formula for this, but simply multiply the longitude circle by the difference between two latitudes...

In [ ]:
import math
Earth = 6370000
dlat = math.pi*Earth/len(zonal_toa.lat)
dlon = 2*math.pi*Earth*np.cos(np.deg2rad(zonal_toa.lat))

Now that we know the area of each latitudinal circle, we can do the integral (and dividing by 10^15 we want PetaWatt not Watt!)

In [ ]:
global_transport = (np.cumsum(zonal_toa-zonal_toa.mean()))*dlon*dlat/10**15
global_transport.plot()
plt.title("Poleward Heat Transport (PW)")

## Time evolving radiation
Is TOA net radiation changing in time? Do you expect it to be increasing, decreasing or being stable?
We can compute its time evolution, don't forget to add the area weigthing!

In [ ]:
area_weights = np.cos(np.deg2rad(net_toa.lat))
net_toa.weighted(area_weights).mean(dim=['lon','lat']).plot()

As we might have expected, the radiation on Earth is slowly increasing in time. But you can see that the climate change signal is small, about 1 W/m2. Considering that TOA radiation is about 240 W/m2, is a demonstration that a small imbalance can lead to significant change to the system

- **What are the two negative spikes in radiation, where for one year the Earth suddenly saw a decrease in net radiation?**

# Cloud Radiative forcing

To better understand the radiation patterns, we might want to have a look at the cloud distribution. In model and data this is usually expresse ad the percentage of each grid point covered by clouds at a given time

In [ ]:
cloud = xr.open_mfdataset('ussp/clt*EC-Earth*.nc')

In [ ]:
cloud['clt'].sel(time=slice('2010-01-01', '2014-12-31')).mean(dim='time').plot()

Does this plot some plot we already saw in the past? Indeed, the radiation budget is strongly affected by the cloud distribution. It is very important thus to determinate the contribution of the cloud to the radiative budget. This can be easily done by estimating the CRF, the cloud radiative forcing. In real world this is not extremely easy, but in model is fairly simple since radiative schemes of climate models also save the so-called clear-sky variables, which are variables were the radiative computation is done in the ideal configuration where no cloud are present. Therefore, using such clear-sky variables, we could estimate the CRF for longwave and shorwave radiation

In [ ]:
long_crf = (dataset['rlutcs']-dataset['rlut']).sel(time=slice('2010-01-01', '2014-12-31')).mean(dim='time')
long_crf.plot()

In [ ]:
short_crf = (dataset['rsutcs']-dataset['rsut']).sel(time=slice('2010-01-01', '2014-12-31')).mean(dim='time')
short_crf.plot()

- **Could you plot and evaluate the globally averaged net cloud radiative forcing, i.e. the combination of longwave and shortwave radiation?** What is the net effect of clouds on the Earth's planet?

## Model biases
To conclude this part, we just do a short analysis to remind us that climate models are great, but they have biases. 
We therefore compare the net_toa radiation we computed before to some satellite based measurement, specifically the **CERES-EBAF** dataset

In [ ]:
ceres = xr.open_dataset('ussp/CERES-2020-2022.nc')
ceres

We need to compare the two datasets on the same period, to have a honest comparison. 

In [ ]:
right_ece = net_toa.sel(time=slice('2001-01-01', '2014-12-31')).mean(dim='time')
right_ceres = ceres['toa_net_all_mon'].sel(time=slice('2001-01-01', '2014-12-31')).mean(dim='time')
bias = right_ece - right_ceres
bias.plot()

In [ ]:
bias.weighted(area_weights).mean().values

Although the bias is really small - it is not just luck, but also a consequence of the tuning of the models - there are large biases of multiple W/m2 in different part of the globe. For example EC-Earth3 has a problem in the representation of clouds in the southern ocean, which leads to a weak cloud radiative forcing in that region and has significant consequences in the representation of the sea surface temperatures and sea ice extent in the Southern Hemisphere. In climate models all is connected, so that one bias can have large implications also on other phenomenona!

# Surface Radiation Budget
Now do the same operation with the radiation budget at the surface. However, this is more complex since there are more components. On top of the radiative heat fluxes, we also have the sensible and latent heat fluxes. 

We need to load also those fluxes, to properly close the budget

In [ ]:
heatfluxes = xr.open_mfdataset('ussp/h*EC-Earth*.nc', join="override", compat='override')

In [ ]:
radiation = xr.merge([heatfluxes, dataset])
radiation

- **Now build your surface radiation flux. You will need six variable from above, which ones?**
- **For your free time: if you compute the meridional integral of the surface heat fluxes, you will obtain the atmospheric heat transport!**